In [ ]:
  !pip install torch==1.9.0+cu102 torchtext==0.10.0 torchvision==0.10.0+cu102 torch-optimizer==0.1.0 -f https://download.pytorch.org/whl/torch/ -f https://download.pytorch.org/whl/torchvision/
  !git clone https://github.com/openai/CLIP
  # !pip install taming-transformers
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf clipit
  !git clone https://github.com/mfrashad/clipit.git
  !pip install ftfy regex tqdm omegaconf pytorch-lightning
  !pip install kornia==0.6.1
  !pip install imageio-ffmpeg   
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  # ClipDraw deps
  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom

  !pip install fastapi
  !pip install python-multipart>=0.0.5
  !pip install pyngrok uvicorn

  !pip3 install opencv-contrib-python
  
  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  # !ls
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  
  !mkdir -p steps
  !mkdir -p models


Looking in links: https://download.pytorch.org/whl/torch/, https://download.pytorch.org/whl/torchvision/
     |███████████████████████▊        | 616.6 MB 1.3 MB/s eta 0:02:47ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 319, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 128, in resolve
    requirements, max_rounds=try_to_avoid_resolution_too_deep
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 473, in resolve
    state = resolution.resolve(requirements, max_rou

In [ ]:
import sys
import IPython
import os


sys.path.append("clipit")
result_msg = "setup complete"
if not os.path.isfile("first_init_complete"):
  # put stuff in here that should only happen once
  !wget https://github.com/Saafke/FSRCNN_Tensorflow/raw/master/models/FSRCNN_x3.pb
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu, and then run Setup again"


In [ ]:
import clipit
import torch
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, File, UploadFile, Form, BackgroundTasks
from fastapi.responses import FileResponse
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import cv2

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}

class GenerateReqBody(BaseModel):
  prompts: str = "Underwater World"

@app.post("/generate")
async def generate(
        reqBody: GenerateReqBody
    ):
    print(reqBody.prompts)
    torch.cuda.empty_cache()
    clipit.reset_settings()
    reqBody.prompts = reqBody.prompts + " low poly"

    clipit.add_settings(prompts=reqBody.prompts,
                        iterations=100,
                        aspect="square",
                        quality="draft",
                        scale=2,
                        use_pixeldraw=False,
                        use_clipdraw=False,
                        make_video=False)
    
    settings = clipit.apply_settings()
    clipit.do_init(settings)
    clipit.do_run(settings)

    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "FSRCNN_x3.pb"
    sr.readModel(path)
    sr.setModel("fsrcnn", 3) 
    img = cv2.imread('output.png')
    result = sr.upsample(img)
    cv2.imwrite('output_inc.png', result)

    return FileResponse('output_inc.png', media_type="image/png")
  

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print('Doc URL:', ngrok_tunnel.public_url+'/docs')
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Output hidden; open in https://colab.research.google.com to view.